In [ ]:
# Extract features from the mask image
# Cell type
#0. background
#1. neoplastic
#2. inflammatory
#3. soft tissue
#4. dead
#5. epithelial

In [ ]:
import math
import numpy as np
import skimage
import matplotlib.pyplot as plt
import histomicstk as htk
import matplotlib.patches as mpatches

In [ ]:
# Set source files
source_dir = '/mnt/storage2/PanNuke/fold1/'
tissue_dir = 'Breast/'
images = source_dir+tissue_dir+'images/img_Breast_1_00000.png'
# masks = source_dir+tissue_dir+'sem_masks/sem_Breast_1_00000.png'
masks = '/home/slee297/sem_masks/test.png'

In [ ]:
# Read images
raw_image = skimage.io.imread(images)
raw_mask = skimage.io.imread(masks)

In [ ]:
# Confirm original images
fig, ax = plt.subplots(1, 2)
ax[0].imshow(raw_image)
ax[1].imshow(raw_mask)

In [ ]:
# Perform color normalization
ref_image_file = ('https://data.kitware.com/api/v1/file/'
                  '57718cc28d777f1ecd8a883c/download')  # L1.png
im_reference = skimage.io.imread(ref_image_file)[:, :, :3]
mean_ref, std_ref = htk.preprocessing.color_conversion.lab_mean_std(im_reference)
im_nmzd = htk.preprocessing.color_normalization.reinhard(raw_image, mean_ref, std_ref)

In [ ]:
# Perform color deconvolution
# create stain to color map
stainColorMap = {
    'hematoxylin': [0.65, 0.70, 0.29],
    'eosin':       [0.07, 0.99, 0.11],
    'dab':         [0.27, 0.57, 0.78],
    'null':        [0.0, 0.0, 0.0],
}

# specify stains of input image
stain_1 = 'hematoxylin'   # nuclei stain
stain_2 = 'eosin'         # cytoplasm stain
stain_3 = 'null'          # set to null of input contains only two stains

# create stain matrix
W = np.array([stainColorMap[stain_1],
              stainColorMap[stain_2],
              stainColorMap[stain_3]]).T

# perform standard color deconvolution
im_stains = htk.preprocessing.color_deconvolution.color_deconvolution(im_nmzd, W).Stains
im_intensity = im_stains[:, :, 0]

In [ ]:
# Confirm original, normalized, and deconvolved images
fig, ax = plt.subplots(1, 3)
ax[0].imshow(raw_image)
ax[1].imshow(im_nmzd)
ax[2].imshow(im_intensity)

In [ ]:
# Change to labels
im_label = skimage.measure.label(raw_mask)

In [ ]:
# find morphometry features
morphometry_features = htk.features.compute_morphometry_features(im_label)

In [ ]:
# find FSD features
FSD_features = htk.features.compute_fsd_features(im_label)

In [ ]:
# find gradient features
gradient_features = htk.features.compute_gradient_features(im_label, im_intensity)

In [ ]:
# find Haralick features
Haralick_features = htk.features.compute_haralick_features(im_label, im_intensity)

In [ ]:
# find intensity features
intensity_features = htk.features.compute_intensity_features(im_label, im_intensity)

In [ ]:
# find all nuclei features
nuclei_features = htk.features.compute_nuclei_features(im_label, im_intensity)

In [ ]:
# find label numbers
label_list = np.unique(im_label)

In [ ]:
# find mask number from labels
mask_list = []
for i in label_list:
    if i != 0:
        mask_label = np.unique(raw_mask[np.where(im_label == label_list[i])])
        if len(mask_label) == 1:
            mask_list.append(mask_label[0])    

In [ ]:
# add a column for mask number; note that the mask number is annotated tissue number
nuclei_features.insert(1, "Mask", mask_list, True)